In [1]:
import ndjson
import os
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter

# Read NDJSON file with keypoints recorded by Labelbox
ndjson_file = "../../ground_truth/Outdoor/court_keypoints/Outdoor.ndjson"  # NDJSON file path
with open(ndjson_file, "r") as f:
    data = ndjson.load(f)

# Key point names and corresponding court coordinate system coordinates
keypoint_mapping = {
    "rku_0": [0, 0],         
    "rku_1": [0, 1105],
    "rku_10": [1505, 0],       
    "rku_11": [1505, 1105]    
}

# Extract key point coordinates on the image from NDJSON
points_image = []
points_court = []

annotations = data[0]["projects"]["cm7u51amc06wh07woexn2g3g4"]["labels"][0]["annotations"]["objects"]
for annotation in annotations:
    key_name = annotation["value"]
    if key_name in keypoint_mapping:
        x, y = annotation["point"]["x"], annotation["point"]["y"]
        points_image.append([x, y])
        points_court.append(keypoint_mapping[key_name]) 

# Convert to numpy array
points_image = np.array(points_image, dtype=np.float32)
points_court = np.array(points_court, dtype=np.float32)

# Calculate homography matrix
H, status = cv2.findHomography(points_image, points_court)

print("Homography Matrix:\n", H)

Homography Matrix:
 [[ 8.67168311e-01 -1.64447031e+00 -4.12433240e+02]
 [-1.33168361e-01 -3.70660422e+00  4.70488602e+03]
 [-1.02627370e-05 -1.70701600e-03  1.00000000e+00]]


In [2]:
def homographic_transformation(x, y, H):
    """Transform the coordinates on the image into the Court coordinate system by homographic transformation"""
    point = np.array([x, y, 1.0]).reshape(3, 1)  # Extended for homographic conversion
    transformed_point = np.dot(H, point)
    transformed_point /= transformed_point[2]  # Normalization
    return transformed_point[0][0], transformed_point[1][0]  # (x', y')

# Court range (using keypoint_mapping)
min_x, min_y = keypoint_mapping["rku_0"]
max_x, max_y = keypoint_mapping["rku_11"]


In [ ]:
# Input folder (BoT-SORT output)
input_dir = "../../output/CAMELTrack_outputs/Outdoor"

# Output folder (court coordinate data)
output_dir_court = "../../output/CAMELTrack_outputs/Outdoor/transformed"
os.makedirs(output_dir_court, exist_ok=True)

# Output folder (filtered bbox data)
output_dir_bbox = "../../output/CAMELTrack_outputs/Outdoor/filtered_MOT"
os.makedirs(output_dir_bbox, exist_ok=True)

# Get MOT files
mot_files = glob.glob(os.path.join(input_dir, "*.txt"))


# Paint extension x coordinate & y boundary
paint_line_y    = 1002.5
paint_x_min     = 502.5
paint_x_max     = 1002.5

for mot_file in mot_files:
    df = pd.read_csv(
        mot_file,
        header=None,
        names=["frame_id","id","x","y","width","height","conf","class","visibility","empty"],
        sep=","
    )

    # Bottom center → Homograph conversion
    df["bottom_center_x"] = df["x"] + df["width"] / 2
    df["bottom_center_y"] = df["y"] + df["height"]
    df[["court_x","court_y"]] = df.apply(
        lambda r: homographic_transformation(r["bottom_center_x"], r["bottom_center_y"], H),
        axis=1, result_type="expand"
    )

    # 2-1) Track-by-track filter
    valid_ids = []
    for track_id in df["id"].unique():
        grp = df[df["id"] == track_id]
        total = grp["frame_id"].nunique()

        # (1) In-court continuity filter
        inside = grp[
            (grp["court_x"]>=min_x)&(grp["court_x"]<=max_x)&
            (grp["court_y"]>=min_y)&(grp["court_y"]<=max_y)
        ]
        frames = sorted(inside["frame_id"].unique())
        max_run = cur = 0; prev = None
        for f in frames:
            cur = cur+1 if prev is not None and f==prev+1 else 1
            max_run = max(max_run, cur); prev = f

        # (2)&(3) Endline area filter
        c1 = (grp["court_y"].between(-300, 100) & grp["court_x"].between(0, 1505)).sum()
        c2 = ((grp["court_y"] > paint_line_y) &
              ((grp["court_x"] < paint_x_min)|(grp["court_x"] > paint_x_max))).sum()

        if max_run >= 10 and c1 <= total/2 and c2 <= total/2:
            valid_ids.append(track_id)

    df = df[df["id"].isin(valid_ids)].dropna()
    df["frame_id"] = df["frame_id"].astype(int)
    df["id"]       = df["id"].astype(int)
    
    # 2-2) Detection level filter: Exclude detections that are more than 3 m (300 cm) away from the court
    buffer_cm = 300  # 3m = 300cm
    df = df[
        (df["court_x"] >= min_x - buffer_cm) &
        (df["court_x"] <= max_x + buffer_cm) &
        (df["court_y"] >= min_y - buffer_cm) &
        (df["court_y"] <= max_y + buffer_cm)
    ]


    # Calculate the frame range of the entire video
    global_min = df["frame_id"].min()
    global_max = df["frame_id"].max()

    # If frame numbers 1 to global_max exist, the total number of frames is global_max
    total_video_frames = global_max

    # ID Switch Detection and Merging
    first_frame_global = global_min
    orig_ids = df[df["frame_id"] == first_frame_global]["id"].unique()
    new_ids  = sorted([i for i in df["id"].unique() if i not in orig_ids])
    # if mot_file == "../../output/CAMELTrack_outputs/Outdoor/IMG_0107_1.txt":
    #     print(new_ids)

    total_frames = df["frame_id"].max()

    for new in new_ids:
        frames_new = set(df[df["id"] == new]["frame_id"].unique())
        first_n    = min(frames_new)
        if first_n <= first_frame_global:
            df = df[df["id"] != new]
            continue

        cands = []
        for orig in orig_ids:
            # Taken only from the inside of the court frame
            inside_orig = set(
                df[
                    (df["id"] == orig) &
                    (df["court_x"].between(min_x, max_x)) &
                    (df["court_y"].between(min_y, max_y))
                ]["frame_id"]
            )
            inside_new  = set(
                df[
                    (df["id"] == new) &
                    (df["court_x"].between(min_x, max_x)) &
                    (df["court_y"].between(min_y, max_y))
                ]["frame_id"]
            )
            
            overlap = len(inside_orig & inside_new)
            if overlap >= 50:
                continue

            union_frames = inside_orig | inside_new
            missing = total_frames - len(union_frames)
            cost    = overlap + missing
            cands.append((orig, cost, overlap, missing))

        if not cands:
            continue

        best_orig, cost, overlap, missing = min(cands, key=lambda x: x[1])

        # Leave orig as it is and delete new for overlapping periods
        # → First, rename new to orig
        df.loc[df["id"] == new, "id"] = best_orig
        # → Then delete duplicate lines (in frames where both orig:new exist, the first orig remains)
        df = df.drop_duplicates(subset=["frame_id","id"], keep="first")

        print(f"Merge ID Switch: new={new} → orig={best_orig} "
              f"(cost={cost}, overlap={overlap}, missing={missing})")
    # --- So far ID Switch detection and merging ---
    
    # # --- BBOX 線形補完 & 末端外挿 ---
    df_bbox = df[["frame_id","id","x","y","width","height","conf","class","visibility","empty"]]
    # interp_list = []

    # # トラックごとに処理
    # for tid, grp in df_bbox_raw.groupby("id"):
    #     # 全フレームで再インデックス化
    #     grp = grp.set_index("frame_id").sort_index()
    #     full_idx = np.arange(global_min, global_max + 1)
    #     grp = grp.reindex(full_idx)

    #     # 各列の補完
    #     for col in ["x","y","width","height"]:
    #         values = grp[col].values.astype(float)
    #         # 線形補間（内部）
    #         idx = np.arange(len(values))
    #         valid = ~np.isnan(values)
    #         if valid.sum() >= 2:
    #             grp[col] = np.interp(idx, idx[valid], values[valid])
    #             # 先頭外挿
    #             first_valid = idx[valid][0]
    #             second_valid = idx[valid][1]
    #             slope_pre = (grp[col].iloc[second_valid] - grp[col].iloc[first_valid]) / (second_valid - first_valid)
    #             for i in range(0, first_valid):
    #                 grp[col].iat[i] = grp[col].iat[first_valid] - slope_pre * (first_valid - i)
    #             # 末端外挿
    #             last_valid = idx[valid][-1]
    #             penult_valid = idx[valid][-2]
    #             slope_post = (grp[col].iloc[last_valid] - grp[col].iloc[penult_valid]) / (last_valid - penult_valid)
    #             for i in range(last_valid + 1, len(values)):
    #                 grp[col].iat[i] = grp[col].iat[last_valid] + slope_post * (i - last_valid)
    #         else:
    #             # データ不足時は前後埋め
    #             grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')

    #     # その他列は直近値で前後埋め
    #     for col in ['conf','class','visibility','empty']:
    #         grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
    #     grp['id'] = tid
    #     grp = grp.reset_index().rename(columns={'index':'frame_id'})
    #     interp_list.append(grp)

    # df_bbox = pd.concat(interp_list, ignore_index=True)


    # --- 出力用データ分割・保存 ---
    df_court = df[["frame_id","id","court_x","court_y"]]

    rel_path  = os.path.relpath(mot_file, input_dir)
    out_court = os.path.join(output_dir_court, rel_path)
    out_bbox  = os.path.join(output_dir_bbox,  rel_path)
    os.makedirs(os.path.dirname(out_court), exist_ok=True)
    os.makedirs(os.path.dirname(out_bbox), exist_ok=True)

    df_court.to_csv(out_court, index=False, header=None, sep=",")
    df_bbox .to_csv(out_bbox,  index=False, header=None, sep=",")
    print(f"Processed: {mot_file} → {out_court}, {out_bbox}")

print(f"すべてのファイルを処理しました: COURT={output_dir_court}, BBOX={output_dir_bbox}")



/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/15223

Processed: ../../CAMELTrack_outputs/Outdoor/IMG_0104_1.txt → ../../CAMELTrack_outputs/Outdoor/transformed/IMG_0104_1.txt, ../../CAMELTrack_outputs/Outdoor/filtered_MOT/IMG_0104_1.txt


/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/15223

Processed: ../../CAMELTrack_outputs/Outdoor/IMG_0104_2.txt → ../../CAMELTrack_outputs/Outdoor/transformed/IMG_0104_2.txt, ../../CAMELTrack_outputs/Outdoor/filtered_MOT/IMG_0104_2.txt
Merge ID Switch: new=14 → orig=8 (cost=29, overlap=20, missing=9)
Processed: ../../CAMELTrack_outputs/Outdoor/IMG_0104_3.txt → ../../CAMELTrack_outputs/Outdoor/transformed/IMG_0104_3.txt, ../../CAMELTrack_outputs/Outdoor/filtered_MOT/IMG_0104_3.txt


/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/15223

...

/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/15223

Merge ID Switch: new=14 → orig=9 (cost=35, overlap=8, missing=27)
Processed: ../../CAMELTrack_outputs/Outdoor/IMG_0110_3.txt → ../../CAMELTrack_outputs/Outdoor/transformed/IMG_0110_3.txt, ../../CAMELTrack_outputs/Outdoor/filtered_MOT/IMG_0110_3.txt


/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/15223

Merge ID Switch: new=15 → orig=5 (cost=33, overlap=10, missing=23)
Processed: ../../CAMELTrack_outputs/Outdoor/IMG_0107_7.txt → ../../CAMELTrack_outputs/Outdoor/transformed/IMG_0107_7.txt, ../../CAMELTrack_outputs/Outdoor/filtered_MOT/IMG_0107_7.txt
すべてのファイルを処理しました: COURT=../../CAMELTrack_outputs/Outdoor/transformed, BBOX=../../CAMELTrack_outputs/Outdoor/filtered_MOT


/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/1522369667.py:179: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grp[col] = grp[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_1441258/15223

In [ ]:


import os
import numpy as np
import re
import csv

# 入力フォルダの設定（MOT形式のtxtと対応するCSV）
mot_folder = '../../ground_truth/Outdoor/MOT_files'
csv_folder = '../../ground_truth/Outdoor/delimitation_frames'

# 出力先のベースフォルダを作成し、さらに free_throw と check_ball のサブフォルダを作成
base_output_folder = os.path.join(mot_folder, 'split_transformed')
free_throw_folder = os.path.join(base_output_folder, 'free_throw')
check_ball_folder = os.path.join(base_output_folder, 'check_ball')
os.makedirs(free_throw_folder, exist_ok=True)
os.makedirs(check_ball_folder, exist_ok=True)

# MOTファイルごとに処理
for mot_file in os.listdir(mot_folder):
    if not mot_file.lower().endswith('.txt'):
        continue
    base_name = os.path.splitext(mot_file)[0]  # 例: IMG_0104
    mot_path = os.path.join(mot_folder, mot_file)
    csv_path = os.path.join(csv_folder, base_name + '.csv')
    if not os.path.exists(csv_path):
        print(f"[警告] {mot_file} に対応するCSVファイルが見つかりません。")
        continue

    # ① MOTファイルを読み込み、各bboxの下辺中点をホモグラフィ変換
    # 出力レコードは (frame, id, court_x, court_y)
    transformed_records = []  
    with open(mot_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # カンマ区切りまたはスペース区切りに対応
            if ',' in line:
                parts = [p.strip() for p in line.split(',')]
                try:
                    # 0始まりの場合、+1
                    frame_num = int(float(parts[0])) + 1
                except ValueError:
                    continue
            else:
                parts = line.split()
                try:
                    frame_num = int(float(parts[0]))
                except ValueError:
                    continue
            try:
                track_id = int(float(parts[1]))
                x, y, w, h = map(float, parts[2:6])
            except ValueError:
                continue
            
            # bboxの下辺中点を計算： (x + w/2, y + h)
            bottom_mid_x = x + w / 2.0
            bottom_mid_y = y + h
            
            # ホモグラフィ変換
            court_x, court_y = homographic_transformation(bottom_mid_x, bottom_mid_y, H)
            
            transformed_records.append((frame_num, track_id, court_x, court_y))
    
    # ② CSVファイルの読み込み（全行読み込む）
    csv_rows = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if row:
                csv_rows.append(row)
    
    if len(csv_rows) % 2 != 0:
        print(f"[警告] {csv_path} の行数が奇数です。最後の行は無視されます。")
    
    # ③ 背番号マッピングはCSVの最初に記載されているため、最初に全体の背番号マッピングを取得
    global_jersey_map = {}
    for row in csv_rows:
        if len(row) >= 2 and row[1].strip() != "":
            jersey_str = row[1].strip()  # 例："1:10, 3: 6, 4: 4, 5: 12, 6: 9, 8: 11"
            for part in jersey_str.split(','):
                subparts = part.split(':')
                if len(subparts) == 2:
                    try:
                        global_jersey_map[int(subparts[0].strip())] = subparts[1].strip()
                    except ValueError:
                        continue
            break  # 1行目のみ使用する

    # ④ セグメントごとに処理（CSVは2行ごとに1セグメント）
    segments = []
    for i in range(0, len(csv_rows) - 1, 2):
        row_odd = csv_rows[i]   # マッピング情報が含まれる行（チーム情報も含む場合がある）
        row_even = csv_rows[i + 1]
        
        # CSV第1列からフレーム番号を抽出
        m1 = re.match(r'(\d+)', row_odd[0].strip())
        m2 = re.match(r'(\d+)', row_even[0].strip())
        if not (m1 and m2):
            continue
        start_frame = int(m1.group(1))
        end_frame = int(m2.group(1))
        
        # 背番号マッピングはグローバルなものを使用
        jersey_map = global_jersey_map
        
        # チームマッピングは row_odd の第3列から抽出（例："Offense: 1, 4, 5; Defense: 3, 6, 8"）
        team_map = {}
        if len(row_odd) > 2:
            team_str = row_odd[2].strip()
            if team_str:
                for part in team_str.split(';'):
                    part = part.strip()
                    if part.startswith("Offense"):
                        ids_str = part[len("Offense:"):].strip()
                        for id_item in ids_str.split(','):
                            try:
                                team_map[int(id_item.strip())] = "O"
                            except ValueError:
                                continue
                    elif part.startswith("Defense"):
                        ids_str = part[len("Defense:"):].strip()
                        for id_item in ids_str.split(','):
                            try:
                                team_map[int(id_item.strip())] = "D"
                            except ValueError:
                                continue
        
        # セグメントタイプの判定：row_oddの1列目に "Just before the referee releases the ball" が含まれていれば free_throw、
        # それ以外は check_ball
        seg_label = row_odd[0].strip()
        seg_type = "free_throw" if "Just before the referee releases the ball" in seg_label else "check_ball"
        
        segments.append({
            "start_frame": start_frame,
            "end_frame": end_frame,
            "jersey_map": jersey_map,
            "team_map": team_map,
            "type": seg_type
        })
    
    # ⑤ 各セグメントごとに、対象のレコードを抽出し、フレーム番号を1始まりにリベースし、追加列（例："O10" や "D6"）を付与
    for seg_idx, seg in enumerate(segments, start=1):
        seg_start = seg["start_frame"]
        seg_end = seg["end_frame"]
        jersey_map = seg["jersey_map"]
        team_map = seg["team_map"]
        seg_type = seg["type"]
        
        segment_output = []
        for rec in transformed_records:
            orig_frame, track_id, court_x, court_y = rec
            if seg_start <= orig_frame <= seg_end:
                # セグメント内でフレーム番号を1始まりにリベース
                new_frame = orig_frame - seg_start + 1
                extra = ""
                if track_id in team_map and track_id in jersey_map:
                    extra = team_map[track_id] + jersey_map[track_id]
                segment_output.append((new_frame, track_id, court_x, court_y, extra))
        
        # 出力先フォルダの選択（free_throw または top）
        folder = free_throw_folder if seg_type == "free_throw" else check_ball_folder
        
        # ⑥ 分割後のtxtファイルとして、カンマ区切りで保存（列名は不要）
        out_filename = f"{base_name}_{seg_idx}.txt"
        out_path = os.path.join(folder, out_filename)
        with open(out_path, 'w', newline='') as fout:
            writer = csv.writer(fout)
            for rec in segment_output:
                writer.writerow([rec[0], rec[1], f"{rec[2]:.3f}", f"{rec[3]:.3f}", rec[4]])
        print(f"セグメント {seg_idx} ({seg_type}) を {out_path} に保存しました。")


セグメント 1 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0105_1.txt に保存しました。
セグメント 2 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0105_2.txt に保存しました。
セグメント 3 (free_throw) を ../../ground_truth/Outdoor/MOT_files/split_transformed/free_throw/IMG_0105_3.txt に保存しました。
セグメント 4 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0105_4.txt に保存しました。
セグメント 5 (free_throw) を ../../ground_truth/Outdoor/MOT_files/split_transformed/free_throw/IMG_0105_5.txt に保存しました。
セグメント 6 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0105_6.txt に保存しました。
セグメント 7 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0105_7.txt に保存しました。
セグメント 8 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0105_8.txt に保存しました。
セグメント 9 (check_ball) を ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_010